# Redes Neuronales 2020
## FAMAF, UNC
### Práctico 1

In [10]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

El sistema de dos ecuaciones diferenciales ordinarias definido como:

$$ \dot{C}(t)  = \alpha C(t) - \beta C(t) Z(t) $$

$$ \dot{Z}(t) = -\gamma Z(t) + \delta C(t) Z(t) $$

Es conocido como **Modelo de predadores y prezas de Lotka-Volterra**. La función $C(t)$ modela el número de conejos en un ecosistema dado, y $Z(t)$ la cantidad de zorros del mismo ecosistema.

a) Construí el diagrama de flujo con las herramientas teóricas presentadas en el curso para el caso particular en que los parámetros toman los siguientes valores:

$$ \alpha = 0.1, \beta = 0.02, \gamma = 0.3, \delta = 0.01 $$

Comenzamos por definir una clase para manejar nuestros modelos:

In [37]:
class Model:
    """
    Define un simple modelo usando un sistema
    de ecuaciones de 2 variables.
    Toma como parámetros las dos funciones que deben ser aplicadas
    a la hora de ser llamadas.
    Params: 
        dC_dt, dZ_dt
    """

    def __init__(self, dC_dt, dZ_dt):
        self.dC_dt = dC_dt
        self.dZ_dt = dZ_dt
    
    def __call__(self, c, z):
        return (self.dC_dt(c, z), self.dZ_dt(c, z))

class LotkaVolterra(Model):
    """
    Define el modelo de Lotka Volterra
    utilizando como base Model.
    Params:
        a, b, c, d
    """
    
    def __init__(self, a, b, c, d):
        dC_dt = lambda c, z: (a * c) - (b * c * z)
        dZ_dt = lambda c, z: ((-c) * z) + (d * c * z)
        super(LotkaVolterra, self).__init__(dC_dt, dZ_dt)

In [41]:
# crear sistema de ecuaciones con los parámetros dados

lotkaVolterra = LotkaVolterra(a=.1, b=.02, c=.3, d=0.01)